In [34]:
import pandas as pd
import numpy as np

In [3]:
app_launch = pd.read_csv('../input/app_launch_log.txt',header=None,sep='\t',names=['user_id','launch_day'])
user_activity = pd.read_csv('../input/user_activity_log.txt',header=None,sep='\t',names=['user_id','activity_day','page','video_id','author_id','action_type'])
user_register = pd.read_csv('../input/user_register_log.txt',header=None,sep='\t',names=['user_id','register_day','register_type','device_type'])
video_create = pd.read_csv('../input/video_create_log.txt',header=None,sep='\t',names=['user_id','create_day'])

In [ ]:
data1_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 16)]
data1_feature=
temp_create = video_create[(video_create['create_day'] >= 1) & (video_create['create_day'] <= 16)]
temp_launch = app_launch[(app_launch['launch_day'] >= 1) & (app_launch['launch_day'] <= 16)]
temp_activity = user_activity[(user_activity['activity_day'] >= 1) & (user_activity['activity_day'] <= 16)]

In [6]:
userPre = user_activity[user_activity['activity_day']>=24]
sub = userPre[['user_id']].drop_duplicates()
sub.to_csv("../submission/baseline0611.csv",encoding='utf-8',index=None,header=None)

In [31]:

df1 = pd.DataFrame({'A':[1,1,2,2],'B':[2,2,6,8],'C':[8,7,6,5]})
df2 = pd.DataFrame({'A':[3,1,6,9,56],'B':[4,2,9,10,76],'D':[1,2,3,4,25]})
#df = df1.merge(df2,how='outer',on=['A','B'],left_index=True)
#t = df1.groupby('A').agg({'C':'min'})

#t = df1.groupby(['A'])['C'].min().rename('T').reset_index()


In [36]:
t = df1.groupby('A').agg({'C':'min'})

In [37]:
t

,C
A,
1,7
2,5


In [38]:
df1

,A,B,C
0,1,2,8
1,1,2,7
2,2,6,6
3,2,8,5


In [41]:
t = df1.groupby('A').agg({'A':'mean'})

In [43]:
t = df1.groupby('A').agg({'B':'mode'})

AttributeError: Cannot access callable attribute 'mode' of 'SeriesGroupBy' objects, try using the 'apply' method